### GAP Data Analytics, Data Retrieval

This Jupyter Notebook allows for automation in the process of extracting GitHub statistics relevant for the redistribution of the GAP programming language. To extract data from the PyGithub API, it is first necessary to install the PyGitHub library. This library provides a Python wrapper for the GitHub REST API.

In [ ]:
# Import sys module for various system-specific parameters and functions
# Exclude lines that are already satisfied using the grep search command
import sys
!{sys.executable} -m pip install numpy pandas matplotlib seaborn PyGithub | grep -v 'already satisfied'

# Import required libraries and packages
import requests
import datetime
from github import Github
from bs4 import BeautifulSoup

# Import modules from other project scripts
from utils import get_github_token

### Connecting to GitHub token

Connecting to GitHub and verifying the user GitHub token is done through storing the access token as an environment variable. This way, the access token is not exposed in the script. The function for getting the token is imported from the utils file in the project.

In [ ]:
# Get the GitHub access token and create instance of the GitHub class
github_token = get_github_token()
if github_token:
    g = Github(github_token)

In [ ]:
# Track the rate limit for GitHub compared to calls used, and see when the limit will reset
remaining_requests, request_limit = g.rate_limiting
print(f"Request Limit for API Calls: {request_limit}")
print(f"Remaining Requests for API Calls: {remaining_requests}")

limit_reset_time = g.rate_limiting_resettime
reset_time = datetime.datetime.fromtimestamp(limit_reset_time).strftime('%Y-%m-%d %H:%M:%S')
print(f"Reset Time for API Calls: {reset_time}")

In [ ]:
# GAP packages: Get the number of GAP packages on GitHub
org_name = "gap-packages"
org = g.get_organization(org_name)

# Get the repositories of the GAP packages organisation that are public
repos = org.get_repos(type="public")
print(f"Number of GAP packages fra GAP Respository: {repos.totalCount}")

In [ ]:
# Test web scraping technique to get the count of GAP packages hosted elsewhere on GitHub
# URL of the GAP Packages website to see the packages and where they are hosted
# NB: This is currently not completely accurate due to the way the GAP packages is listed, counts per entity

# Send a GET request to the website where the list of GAP packages is hosted
url = "https://gap-packages.github.io/"
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find the section of the webpage with the packages stored elsewhere on GitHub
section = soup.find("section", id="main-content")
heading = section.find(id="packages-hosted-elsewhere-on-github")
ul = heading.find_next("ul")

# Do not include any child elements that are ul or li, as not to let these increase the count
packages = ul.find_all("li", recursive=False)
count = len(packages)

print(f"Number of GAP packages hosted elsewhere on GitHub: {count}")